In [252]:
import os
import json
import pandas as pd
from openai import OpenAI
from dotenv import load_dotenv

In [ ]:
# Read the CSV file into a Pandas DataFrame
# print(os.getcwd())
calendar_data = pd.read_csv("../data/calendar_data.csv")
print(f"The number of calendar data is {len(calendar_data)}.")

1. Loading the dataset and json
2. Prompt the dataset column and json question to gpt-4 to generate the python function
3. pass the generated funtion and dataframe into the test function
4. save the result into json-ansewer

In [264]:
question_index = 0 # for question test

with open("../data/question.json") as json_file:
    json_data = json.load(json_file)
    print(json_data[question_index])

{'question': 'How many meetings do I have scheduled in total?', 'answer': ''}


In [265]:
print(f"The total number of the question is {len(json_data)}.")
print(f"Selected question is: <{json_data[question_index]['question']}>.")

The total number of the question is 8.
Selected question is: <How many meetings do I have scheduled in total?>.


In [266]:
text = f"""
{json_data[question_index]['question']}
"""
prompt = f"""You are provided a DataFrame ```calendar_data(ID, status, summary, start, end , duration, attendees)```. This dataframe includes all you calenders.
Your task is generate python function to query this dataframe and answer the question. Output a python function object without any prefix instructionn and explaination.

The input have following columns:
- ID: meeting ID;
- status: meeting status, including the following status: cancelled, confirmed, tentative;
- summary: meeting or event topic;
- start: the start date of meeting, date format: YYYY-MM-DD hh:mm:ss-00:00, for example "2024-02-05 12:00:00-00:00";
- end: the start date of meeting, date format: YYYY-MM-DD hh:mm:ss-00:00, for example "2024-02-05 13:00:00-00:00";
- duration: meeting duration (second);
- attendees: people who attend the meeting delimited by '\n' with 1 sentence.

Output using the folowing format:
code language: python
function name: query():
input: calendar_data
output is a python function without any prefix instructionn and explaination.

Today is 2024-03-12 13:02:30-03:20

For example, the output only have the following format:
Sample 1:
def query(calendar_data):
    return calendar_data[0]

Question to be resolved: {text} 
"""

In [267]:
print(prompt)

You are provided a DataFrame ```calendar_data(ID, status, summary, start, end , duration, attendees)```. This dataframe includes all you calenders.
Your task is generate python function to query this dataframe and answer the question. Output a python function object without any prefix instructionn and explaination.

The input have following columns:
- ID: meeting ID;
- status: meeting status, including the following status: cancelled, confirmed, tentative;
- summary: meeting or event topic;
- start: the start date of meeting, date format: YYYY-MM-DD hh:mm:ss-00:00, for example "2024-02-05 12:00:00-00:00";
- end: the start date of meeting, date format: YYYY-MM-DD hh:mm:ss-00:00, for example "2024-02-05 13:00:00-00:00";
- duration: meeting duration (second);
- attendees: people who attend the meeting delimited by '
' with 1 sentence.

Output using the folowing format:
code language: python
function name: query():
input: calendar_data
output is a python function without any prefix instruc

In [268]:

MODEL = "gpt-4"
# MODEL = "gpt-3.5-turbo"

client = OpenAI(
    # This is the default and can be omitted
    api_key=os.environ.get("OPENAI_API_KEY"),
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": prompt,
        }
    ],
    model=MODEL,
)

# verify the output
generated_text = chat_completion.choices[0].message.content
print(generated_text)

def query(calendar_data):
    scheduled_meetings = calendar_data[calendar_data['status'] != 'cancelled']
    return len(scheduled_meetings)


In [269]:
fp = open('generated_code.py', 'w')
# Add package dependency, eg import panda as pd 
fp.write(generated_text)
fp.close()

In [270]:
import importlib
# Import your module
import generated_code

# Must update the function 
importlib.reload(generated_code)
answer = generated_code.query(calendar_data)
print(answer)

2
18


In [ ]:
def query(calendar_data):
    today = '2024-03-12'
    today_events = calendar_data[(calendar_data['start'].str.contains(today)) & (calendar_data['status'] == 'confirmed')]
    return len(today_events)
answer = query(calendar_data)
print(answer)

Save the result into the .json file

In [ ]:
print(json_data)


### Error

MUST self-check the output of llm, until the output is a function without any prefix sentences !!! 

1 "\n"
    should be "\\n"
2 Date
    unexpcted prefix
2. python package, eg 
    def query():
        import panda as pd
